<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_5_%D0%9F%D0%B5%D1%80%D1%81%D0%BE%D0%BD%D0%B8%D1%84%D0%B8%D1%86%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D0%B0%D1%8F_%D0%B8_%D0%BF%D1%80%D0%B5%D0%B4%D0%B8%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B5%D0%B4%D0%B8%D1%86%D0%B8%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyhealth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.

In [2]:
from pyhealth.datasets import MIMIC3Dataset

mimic3_ds = MIMIC3Dataset(
        root="https://storage.googleapis.com/pyhealth/Synthetic_MIMIC-III/",
        tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
        code_mapping={"NDC": ("ATC", {"target_kwargs": {"level": 3}})},
        dev=False
)

mimic3_ds.stat()

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
finish basic patient information parsing : 93.8720805644989s
finish parsing DIAGNOSES_ICD : 66.24897050857544s
finish parsing PROCEDURES_ICD : 62.75838613510132s
finish parsing PRESCRIPTIONS : 175.146240234375s


Mapping codes: 100%|██████████| 49993/49993 [00:51<00:00, 963.81it/s] 



Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 49993
	- Number of visits: 52769
	- Number of visits per patient: 1.0555
	- Number of events per visit in DIAGNOSES_ICD: 9.1038
	- Number of events per visit in PROCEDURES_ICD: 3.2186
	- Number of events per visit in PRESCRIPTIONS: 32.9969



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 49993\n\t- Number of visits: 52769\n\t- Number of visits per patient: 1.0555\n\t- Number of events per visit in DIAGNOSES_ICD: 9.1038\n\t- Number of events per visit in PROCEDURES_ICD: 3.2186\n\t- Number of events per visit in PRESCRIPTIONS: 32.9969\n'

In [3]:
# data format
mimic3_ds.info()


dataset.patients: patient_id -> <Patient>

<Patient>
    - visits: visit_id -> <Visit> 
    - other patient-level info
    
    <Visit>
        - event_list_dict: table_name -> List[Event]
        - other visit-level info
    
        <Event>
            - code: str
            - other event-level info



In [4]:
""" check the information """
mimic3_ds.patients
mimic3_ds.patients['947']
mimic3_ds.patients['947'].visits
mimic3_ds.patients['947'].visits['100999']
mimic3_ds.patients['947'].visits['100999'].get_code_list('DIAGNOSES_ICD')

['40390', '3484', '2989']

In [5]:
from pyhealth.tasks import drug_recommendation_mimic3_fn

mimic3_ds = mimic3_ds.set_task(task_fn=drug_recommendation_mimic3_fn)
# stats info
mimic3_ds.stat()

Generating samples for drug_recommendation_mimic3_fn: 100%|██████████| 49993/49993 [00:01<00:00, 35560.14it/s]


Statistics of sample dataset:
	- Dataset: MIMIC3Dataset
	- Task: drug_recommendation_mimic3_fn
	- Number of samples: 1858
	- Number of patients: 911
	- Number of visits: 1858
	- Number of visits per patient: 2.0395
	- conditions:
		- Number of conditions per sample: 16.0759
		- Number of unique conditions: 2146
		- Distribution of conditions (Top-10): [('4019', 1107), ('4280', 755), ('42731', 717), ('41401', 713), ('25000', 659), ('2724', 431), ('5849', 412), ('5990', 393), ('53081', 373), ('2720', 295)]
	- procedures:
		- Number of procedures per sample: 5.9462
		- Number of unique procedures: 659
		- Distribution of procedures (Top-10): [('3893', 642), ('9904', 495), ('3961', 431), ('9604', 327), ('9671', 313), ('966', 309), ('8856', 272), ('3615', 268), ('8872', 242), ('3722', 200)]
	- drugs:
		- Number of drugs per sample: 21.4769
		- Number of unique drugs: 178
		- Distribution of drugs (Top-10): [('B05X', 1393), ('A06A', 1295), ('A02B', 1232), ('B01A', 1195), ('N02B', 1182), ('N0

"Statistics of sample dataset:\n\t- Dataset: MIMIC3Dataset\n\t- Task: drug_recommendation_mimic3_fn\n\t- Number of samples: 1858\n\t- Number of patients: 911\n\t- Number of visits: 1858\n\t- Number of visits per patient: 2.0395\n\t- conditions:\n\t\t- Number of conditions per sample: 16.0759\n\t\t- Number of unique conditions: 2146\n\t\t- Distribution of conditions (Top-10): [('4019', 1107), ('4280', 755), ('42731', 717), ('41401', 713), ('25000', 659), ('2724', 431), ('5849', 412), ('5990', 393), ('53081', 373), ('2720', 295)]\n\t- procedures:\n\t\t- Number of procedures per sample: 5.9462\n\t\t- Number of unique procedures: 659\n\t\t- Distribution of procedures (Top-10): [('3893', 642), ('9904', 495), ('3961', 431), ('9604', 327), ('9671', 313), ('966', 309), ('8856', 272), ('3615', 268), ('8872', 242), ('3722', 200)]\n\t- drugs:\n\t\t- Number of drugs per sample: 21.4769\n\t\t- Number of unique drugs: 178\n\t\t- Distribution of drugs (Top-10): [('B05X', 1393), ('A06A', 1295), ('A02B

In [6]:
mimic3_ds.samples[17]

{'visit_id': '110765',
 'patient_id': '10402',
 'conditions': [['2639',
   '4821',
   '2720',
   '3051',
   '42731',
   '28521',
   '5601',
   '30500',
   '2724',
   '2767',
   '1985',
   '34590',
   '2763',
   '07054',
   '2866',
   '78900',
   '44023',
   '99649'],
  ['51881',
   '5990',
   '4019',
   '5070',
   '7907',
   '5849',
   '5789',
   '42789',
   'E8798',
   '2800',
   'V5861',
   '2875',
   '5856',
   '99883',
   '4538',
   '78052',
   '4275',
   '73390',
   '20410',
   '2841',
   '43320',
   'V8741',
   'V440',
   '99681',
   '99659',
   'V454',
   '2989',
   '51851',
   '43889',
   '49322',
   'V642',
   '56721',
   '5130',
   '78722',
   '72982']],
 'procedures': [['9604', '5185'],
  ['9604',
   '966',
   '3893',
   '9915',
   '4311',
   '5491',
   '3897',
   '3491',
   '560',
   '3324',
   '9723',
   '8191',
   '0014',
   '4516',
   '9925',
   '9390',
   '9229',
   '8851',
   '9906',
   '8754',
   '415']],
 'drugs': ['J01E',
  'A02B',
  'J01X',
  'N03A',
  'N02A',
  'J

In [7]:
from pyhealth.datasets.splitter import split_by_patient
from pyhealth.datasets import split_by_patient, get_dataloader

# data split
train_dataset, val_dataset, test_dataset = split_by_patient(mimic3_ds, [0.7, 0.15, 0.15])

# create dataloaders (they are <torch.data.DataLoader> object)
train_loader = get_dataloader(train_dataset, batch_size=64, shuffle=True)
val_loader = get_dataloader(val_dataset, batch_size=64, shuffle=True)
test_loader = get_dataloader(test_dataset, batch_size=64, shuffle=True)

In [8]:
from pyhealth.models import Transformer

model = Transformer(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="drugs",
    mode="multilabel",
)

In [9]:
from pyhealth.trainer import Trainer

trainer = Trainer(model=model)
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=17,
    monitor="pr_auc_samples",
)

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2148, 128, padding_idx=0)
    (procedures): Embedding(661, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activation): GELU(approximate='none')
          )
          (i

INFO:pyhealth.trainer:Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(2148, 128, padding_idx=0)
    (procedures): Embedding(661, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0-2): 3 x Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
            (activation): GELU(approximate='none')
  

Metrics: None


INFO:pyhealth.trainer:Metrics: None


Device: cpu


...

INFO:pyhealth.trainer:pr_auc_samples: 0.5553


loss: 0.2953


INFO:pyhealth.trainer:loss: 0.2953


Loaded best model


INFO:pyhealth.trainer:Loaded best model


In [10]:
# option 1: use our built-in evaluation metric
score = trainer.evaluate(test_loader)
print (score)

# option 2: use our pyhealth.metrics to evaluate
from pyhealth.metrics.multilabel import multilabel_metrics_fn
y_true, y_prob, loss = trainer.inference(test_loader)
multilabel_metrics_fn(y_true, y_prob, metrics=["pr_auc_samples"])

Evaluation: 100%|██████████| 5/5 [00:00<00:00, 31.21it/s]


{'pr_auc_samples': 0.5618260452845631, 'loss': 0.28119567036628723}


Evaluation: 100%|██████████| 5/5 [00:00<00:00, 29.00it/s]


{'pr_auc_samples': 0.5618260452845631}